In [1]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [2]:
# import pandas as pd 
# combined_gdf = gpd.GeoDataFrame(pd.concat([n1, f.e,f.e], ignore_index=True))
# combined_gdf.to_file(f'OPEN_FIN.gpkg', driver='GPKG')

In [3]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"spblo.graphml")

In [4]:
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
import os
# folder = os.getcwd().split("\notebooks")[0]
from src.graphbuilder.graph import Graph 
from src.frame_grader.advanced_grade import AdvancedGrader
from src.framebuilder.frame import Frame
import src.utils.helper_funcs as he
from src.indicators.indicator_terr import indicator_territory
from src.indicators.indicator_area import indicator_area, preprocess_service_accessibility
from src.indicators.utils import availability_matrix, create_service_dict
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely
from src.graphbuilder.graph import convert_geometry_from_wkt
import pandas as pd



/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 ['/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages', '/Users/polina/Desktop/github/transport_frames', '/Users/polina/Desktop/github/transport_frames'] 0


## Graph creation

In [5]:
# geocode=81993
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=32636) #  get lo polygon
polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:32636")
local_crs = 32636

In [6]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/notebooks/spblo.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2024-10-07 03:17:26.862 | INFO     | src.graphbuilder.graph:_prepare_attrs:138 - Preparing the graph...
2024-10-07 03:18:19.216 | INFO     | src.graphbuilder.graph:_prepare_attrs:176 - Graph is ready!


## Frame creation

In [7]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))].to_crs(local_crs)

admin_centers = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Ленинградская_область 2/Ленинградская_область_district_admin_centres_18_nodes.geojson')
# admin_centers = admin_centers[admin_centers.is_admin_centre_district==1][['name','geometry']].reset_index()
admin_centers['name'] = admin_centers['name18']
admin_centers = admin_centers[['name','geometry']].reset_index().to_crs(local_crs)


In [8]:
restricted_terr_gdf = gpd.GeoDataFrame({
    'geometry': [ox.geocode_to_gdf('Estonia').geometry[0], ox.geocode_to_gdf('Finland').geometry[0]],
    'mark': [0.0, 0.5]
},crs=4326)

In [9]:
f = Frame(g.graph, regions.to_crs(local_crs), polygon.to_crs(local_crs), admin_centers.to_crs(local_crs),restricted_terr=restricted_terr_gdf)
# f.get_geopackage();

In [11]:
n,e = momepy.nx_to_gdf(f.frame)

## Grading custom territory (frame-based)

In [11]:
neud1 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/Аэродром лодейнопольское поселение.geojson').to_crs(32636)
neud2 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/Аэродром Сиверск .geojson').to_crs(32636)
neud3 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/project Светогорского поселения.geojson').to_crs(32636)
neud3['name'] = 'Светогорское поселение'
neud4 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/project Шлиссельбург.geojson').to_crs(32636)
neud4['name'] = 'Шлиссельбург'


neud = pd.concat([neud1,neud3,neud2,neud4]).reset_index()

In [12]:
neud['layer']='l'
neud['status']='l'

In [13]:
graded_terr = f.grade_territory(neud)

In [14]:
graded_terr

,name,layer,status,geometry,grade
0,Лодейное Поле,l,l,"MULTIPOLYGON (((529877.020 6730827.593, 529893...",5.0
1,Светогорское поселение,l,l,"POLYGON ((278277.965 6784490.204, 279737.375 6...",4.5
2,Сиверский,l,l,"MULTIPOLYGON (((329788.882 6583309.434, 330250...",5.0
3,Шлиссельбург,l,l,"POLYGON ((387861.999 6644712.938, 388202.551 6...",5.0


## Criteria of the territory


In [15]:
settlement_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/lo_gdfs/points.geojson')
settlement_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/lo_gdfs/polygons188.geojson')
railway_stations = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Ленинградская_область 2/Ленинградская_область_railway_stations.geojson')
bus_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Ленинградская_область 2/Ленинградская_область_bus_stops.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Ленинградская_область 2/Ленинградская_область_local_aerodrome.geojson')
ferry = gpd.read_file('/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/lo_gdfs/water_transport_LO.geojson')

In [16]:
# inter = he.prepare_graph(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/inter.graphml'))
# inter = he.convert_list_attr_from_str(inter)
# inter.graph['crs'] = local_crs

# inter = get_intermodal_graph(osm_id=geocode,
#                         keep_routes_geom=True,clip_by_bounds=True)

In [17]:
adj_drive = availability_matrix(g.graph, settlement_points,settlement_points,local_crs=local_crs)
# adj_inter = availability_matrix(inter, settlement_points, settlement_points, local_crs=local_crs)
adj_inter = pd.read_pickle('adj_inter.pickle')

In [18]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,
                          points=settlement_points,
                          polygons=settlement_polygons,
                          r_stops=railway_stations,
                          b_stops=bus_stops,
                          aero=aero,
                          ferry=ferry,
                          adj_mx_drive=adj_drive,
                          adj_mx_inter=adj_inter)

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [19]:
cri

,geometry,name,grade,weight,weight_r_stops,weight_b_stops,weight_ferry,weight_aero,car_access_quartile,public_access_quartile,car_grade,public_transport_grade,overall_assessment
0,"MULTIPOLYGON (((529877.020 6730827.593, 529893...",Лодейное Поле,5.0,0.90,0.35,0.35,0.2,0.0,4,4,2.0,1.8,1.90
1,"POLYGON ((278277.965 6784490.204, 279737.375 6...",Светогорское поселение,4.5,0.35,0.00,0.35,0.0,0.0,4,4,2.0,0.7,1.35
2,"MULTIPOLYGON (((329788.882 6583309.434, 330250...",Сиверский,5.0,0.80,0.35,0.35,0.0,0.1,1,1,5.0,4.0,4.50
3,"POLYGON ((387861.999 6644712.938, 388202.551 6...",Шлиссельбург,5.0,1.00,0.35,0.35,0.2,0.1,1,1,5.0,5.0,5.00


## Indicators of the territory (graph-based)

In [20]:
railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/ЖД остановки.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/ЖД пути.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/Остановки ЛО.geojson')
bus_stops = bus_stops[bus_stops.geometry.notna()]
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/Автобусные маршруты ЛО.geojson')
bus_routes = bus_routes[bus_routes.geometry.apply(lambda geom: geom.geom_type in ['LineString', 'MultiLineString'])]

fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/fuel.geojson')
ferry_terminal = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/water_transport_LO.geojson')
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/airports_local_LO.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
international_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/lo_gdfs/pulkovo.geojson')
# либо:
# international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/Водомы итог (конец).geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/ООПТ ЛО.geojson')

In [21]:
capital=ox.geocode_to_gdf('N27490597',by_osmid=True)

In [22]:
districts = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/polygons18.geojson')
districts['status'] = 'district'
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/polygons188.geojson')
settlements['status'] = 'settlement'
settlements_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/points.geojson')

In [23]:
services = create_service_dict(
                                railway_stations=railway_stations,
                                railway_paths=railway_roads,
                                bus_stops=bus_stops,
                                bus_routes=bus_routes,
                                fuel_stations=fuel_stations,
                                local_aerodrome=local_aerodrome,
                                ferry_terminal=ferry_terminal,
                                international_aerodrome=international_aerodrome,
                                water_objects=water,
                                nature_reserve=nature_reserve,          
                                local_crs=local_crs)

In [24]:
t = indicator_territory(g.graph,neud,services,capital,admin_centers,settlement_points,districts,local_crs,)
t

,name,geometry,to_region_admin_center_km,to_reg_1_km,fuel_stations_accessibility_min,number_of_fuel_stations,local_aerodrome_accessibility_min,number_of_local_aerodrome,international_aerodrome_accessibility_min,number_of_international_aerodrome,...,number_of_bus_stops,number_of_water_objects,water_objects_accessibility_min,number_of_nature_reserve,nature_reserve_accessibility_min,train_path_length_km,number_of_bus_routes,to_nearest_district_center_km,to_nearest_settlement_km,road_density_km/km2
0,Лодейное Поле,"POLYGON ((526409.619 6730335.647, 526410.168 6...",234.836,3.870,0.00,4.0,44.914,0.0,162.969,0.0,...,47.0,65,0.0,0,50.368,10.527,25,6.799,6.563,1.633
1,Светогорское поселение,"POLYGON ((277278.360 6787318.771, 283165.662 6...",189.040,5.091,0.00,1.0,81.347,0.0,139.904,0.0,...,28.0,20,0.0,0,10.467,6.360,5,57.585,5.632,0.615
2,Сиверский,"POLYGON ((326883.854 6582560.561, 326796.404 6...",74.247,5.657,7.95,0.0,13.132,0.0,46.922,0.0,...,67.0,1,0.0,0,0.552,7.441,13,30.958,2.749,2.155
3,Шлиссельбург,"POLYGON ((385097.278 6645877.548, 385211.554 6...",46.505,3.258,0.00,2.0,24.823,0.0,35.485,0.0,...,41.0,130,0.0,0,5.652,9.216,36,6.251,6.251,3.925


## Indicators of the area (graph-based)

In [25]:
#area polygons
country_polygon = ox.geocode_to_gdf('RUSSIA')
country_polygon['status'] = 'country'
country_polygon['layer'] = 'country'

In [26]:
preprocessed_accessibility = preprocess_service_accessibility(settlement_points, services, g.graph, local_crs)

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/pandas/core/construction.py:820: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)


In [27]:
ind_area = indicator_area(g.graph, [settlements, districts, country_polygon], preprocessed_accessibility, services, capital, local_crs, adj_drive, adj_inter)

Processing areas:   0%|          | 0/3 [00:00<?, ?it/s]2024-10-04 17:30:18.656 | INFO     | src.indicators.indicator_area:indicator_area:156 - Calculating service accessibility
2024-10-04 17:30:19.005 | INFO     | src.indicators.indicator_area:indicator_area:175 - Calculating distance to region admin center and federal roads
2024-10-04 17:30:23.232 | INFO     | src.indicators.indicator_area:indicator_area:180 - Calculating train path lengths
2024-10-04 17:30:24.069 | INFO     | src.indicators.indicator_area:indicator_area:189 - Calculating number of bus routes
2024-10-04 17:30:30.730 | INFO     | src.indicators.indicator_area:indicator_area:199 - Calculating reg_1 road lengths
2024-10-04 17:30:32.530 | INFO     | src.indicators.indicator_area:indicator_area:199 - Calculating reg_2 road lengths
2024-10-04 17:30:34.324 | INFO     | src.indicators.indicator_area:indicator_area:208 - Calculating road density
Processing areas:  33%|███▎      | 1/3 [01:21<02:43, 81.63s/it]2024-10-04 17:31:40

In [28]:
ind_area[0]


,name,layer,status,geometry,number_of_railway_stations,railway_stations_accessibility_min,number_of_fuel_stations,fuel_stations_accessibility_min,number_of_ports,ports_accessibility_min,...,international_aerodrome_accessibility_min,number_of_bus_stops,connectivity_drive_min,to_region_admin_center_km,to_reg_1_km,train_path_length_km,number_of_bus_routes,reg1_length_km,reg2_length_km,road_density_km/km2
0,Самойловское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((580049.515 6617067.720, 580225...",2.0,18.171875,2.0,20.156250,0.0,168.125000,...,197.000000,68.0,220.9920,271.457,6.088,19.068,12,18.632,18.632,0.113
1,Большедворское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((573514.444 6620389.851, 573579...",8.0,13.132812,1.0,19.656250,0.0,145.000000,...,173.875000,50.0,197.8690,251.336,18.029,23.824,16,32.324,32.324,0.142
2,Пикалевское городское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((564707.328 6594903.594, 564341...",1.0,6.019531,2.0,0.960938,0.0,152.125000,...,181.000000,59.0,204.9890,257.187,0.084,14.023,20,14.456,14.456,1.168
3,Борское сельское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((560629.509 6557332.762, 560642...",0.0,29.046875,1.0,15.476562,0.0,157.500000,...,186.500000,44.0,210.4410,263.421,26.426,0.000,12,0.000,0.000,0.106
4,Бокситогорское городское поселение,Бокситогорский муниципальный район,settlement,"MULTIPOLYGON (((560659.317 6585430.874, 559919...",0.0,13.554688,2.0,5.132812,0.0,144.500000,...,173.375000,34.0,197.3370,257.706,18.846,0.000,15,7.097,7.097,0.281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Фёдоровское городское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((360431.648 6619592.469, 360583...",2.0,5.506836,0.0,11.023438,0.0,46.890625,...,24.226562,21.0,111.2180,40.182,7.612,12.336,3,0.000,0.000,1.312
184,Нурминское сельское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((387941.398 6612286.590, 388009...",2.0,1.241211,0.0,7.613281,0.0,52.062500,...,44.281250,15.0,122.3930,67.299,11.458,9.254,4,0.000,0.000,0.348
185,Красноборское городское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((363803.737 6614904.469, 363821...",2.0,3.471680,1.0,7.248047,0.0,33.218750,...,25.453125,29.0,111.6755,40.762,3.379,10.498,3,29.583,29.583,0.952
186,Трубникоборское сельское поселение,Тосненский муниципальный район,settlement,"MULTIPOLYGON (((392924.072 6551507.217, 393174...",7.0,8.468750,4.0,7.199219,0.0,82.531250,...,72.500000,44.0,146.3060,102.241,8.596,17.975,4,77.274,77.274,0.198
